# Google Vertex AI 搜索

>[Google Vertex AI 搜索](https://cloud.google.com/enterprise-search)（以前称为 `Generative AI App Builder` 上的 `Enterprise Search`）是 `Google Cloud` 提供的 [Vertex AI](https://cloud.google.com/vertex-ai) 机器学习平台的一部分。
>
> `Vertex AI 搜索`使组织能够为客户和员工快速构建由生成式 AI 提供支持的搜索引擎。它基于各种 `Google 搜索`技术，包括语义搜索。语义搜索通过使用自然语言处理和机器学习技术来推断内容中的关系以及用户查询输入的意图，从而帮助提供比传统的基于关键词的搜索技术更相关的结果。Vertex AI 搜索还受益于 Google 在理解用户搜索方式方面的专业知识，并考虑内容相关性来对显示的结果进行排序。

> `Vertex AI 搜索`可在 `Google Cloud Console` 中使用，也可通过 API 进行企业工作流集成。

本笔记本演示了如何配置 `Vertex AI 搜索`以及如何使用 Vertex AI 搜索 [检索器](/docs/concepts/retrievers)。Vertex AI 搜索检索器封装了 [Python 客户端库](https://cloud.google.com/generative-ai-app-builder/docs/libraries#client-libraries-install-python)，并使用它来访问 [搜索服务 API](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.services.search_service)。

有关 `VertexAISearchRetriever` 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/google_community/vertex_ai_search/langchain_google_community.vertex_ai_search.VertexAISearchRetriever.html)。

### 集成详情

import {ItemTable} from "@theme/FeatureTables";

<ItemTable category="document_retrievers" item="VertexAISearchRetriever" />

## 设置

### 安装

您需要安装 `langchain-google-community` 和 `google-cloud-discoveryengine` 包才能使用 Vertex AI 搜索检索器。

In [ ]:
%pip install -qU langchain-google-community google-cloud-discoveryengine

### 配置对 Google Cloud 和 Vertex AI Search 的访问权限

Vertex AI Search 自 2023 年 8 月起已普遍可用，无需进行许可列表设置。

在使用检索器之前，您需要完成以下步骤：

#### 创建搜索引擎并填充非结构化数据存储

- 请遵循 [Vertex AI Search入门指南](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search) 中的说明，设置 Google Cloud 项目和 Vertex AI Search。
- [使用 Google Cloud Console 创建非结构化数据存储](https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es#unstructured-data)
  - 使用 `gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs` Cloud Storage 文件夹中的示例 PDF 文档进行填充。
  - 请确保使用 `Cloud Storage (without metadata)` 选项。

#### 设置访问 Vertex AI Search API 的凭据

Vertex AI Search 检索器使用的 [Vertex AI Search 客户端库](https://cloud.google.com/generative-ai-app-builder/docs/libraries) 提供高级语言支持，用于以编程方式连接到 Google Cloud。
客户端库支持 [应用程序默认凭据 (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials)；这些库会在一组预定义的地点查找凭据，并使用这些凭据向 API 进行身份验证请求。
通过 ADC，您可以在各种环境（如本地开发或生产环境）中为您的应用程序提供凭据，而无需修改应用程序代码。

如果在 [Google Colab](https://colab.google) 中运行，请使用 `google.colab.google.auth` 进行身份验证；否则，请遵循[支持的方法](https://cloud.google.com/docs/authentication/application-default-credentials)之一，以确保您的应用程序默认凭据已正确设置。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 配置和使用 Vertex AI Search 检索器

Vertex AI Search 检索器在 `langchain_google_community.VertexAISearchRetriever` 类中实现。`get_relevant_documents` 方法返回一个 `langchain.schema.Document` 文档列表，其中每个文档的 `page_content` 字段都填充了文档内容。
根据 Vertex AI Search 中使用的数据类型（网站、结构化或非结构化），`page_content` 字段的填充方式如下：

- 具有高级索引的网站：与查询匹配的`提取式答案`。`metadata` 字段填充了从从中提取片段或答案的文档的元数据（如果有）。
- 非结构化数据源：与查询匹配的`提取式片段`或`提取式答案`。`metadata` 字段填充了从从中提取片段或答案的文档的元数据（如果有）。
- 结构化数据源：包含从结构化数据源返回的所有字段的字符串 JSON。`metadata` 字段填充了文档的元数据（如果有）

#### 提取式答案和提取式片段

提取式答案是随每个搜索结果返回的逐字文本。它直接从原始文档中提取。提取式答案通常显示在网页的顶部附近，为最终用户提供与查询相关的简明答案。提取式答案可用于网站和非结构化搜索。

提取式片段是随每个搜索结果返回的逐字文本。提取式片段通常比提取式答案更详细。提取式片段可用作查询的答案，并可用于执行后处理任务，以及作为大型语言模型的输入来生成答案或新文本。提取式片段可用于非结构化搜索。

有关提取式片段和提取式答案的更多信息，请参阅 [产品文档](https://cloud.google.com/generative-ai-app-builder/docs/snippets)。

注意：提取式片段需要启用 [Enterprise 版本](https://cloud.google.com/generative-ai-app-builder/docs/about-advanced-features#enterprise-features) 功能。

在创建检索器实例时，您可以指定控制访问哪些数据存储以及如何处理自然语言查询的多个参数，包括提取式答案和片段的配置。

#### 必需的参数是：

- `project_id` - 您的 Google Cloud 项目 ID。
- `location_id` - 数据存储的位置。
  - `global` (默认)
  - `us`
  - `eu`

以下其中一项：
- `search_engine_id` - 您想使用的搜索应用的 ID。（混合搜索必需）
- `data_store_id` - 您想使用的数据存储的 ID。

`project_id`、`search_engine_id` 和 `data_store_id` 参数可以在检索器的构造函数中显式提供，或者通过环境变量 `PROJECT_ID`、`SEARCH_ENGINE_ID` 和 `DATA_STORE_ID` 提供。

您还可以配置多个可选参数，包括：

- `max_documents` - 用于提供提取式片段或提取式答案的最大文档数
- `get_extractive_answers` - 默认情况下，检索器配置为返回提取式片段。
  - 将此字段设置为 `True` 以返回提取式答案。仅当 `engine_data_type` 设置为 `0`（非结构化）时使用
- `max_extractive_answer_count` - 每个搜索结果返回的最大提取式答案数。
  - 每个结果最多返回 5 个答案。仅当 `engine_data_type` 设置为 `0` (非结构化) 时使用。
- `max_extractive_segment_count` - 每个搜索结果返回的最大提取式片段数。
  - 目前每个结果返回一个片段。仅当 `engine_data_type` 设置为 `0` (非结构化) 时使用。
- `filter` - 基于数据存储中文档关联的元数据的搜索结果筛选表达式。
- `query_expansion_condition` - 指定查询展开应在何种条件下发生。
  - `0` - 未指定的查询展开条件。在此情况下，服务器行为默认为禁用。
  - `1` - 禁用的查询展开。仅使用精确的搜索查询，即使 SearchResponse.total_size 为零。
  - `2` - 由 Search API 构建的自动查询展开。
- `engine_data_type` - 定义 Vertex AI Search 数据类型
  - `0` - 非结构化数据
  - `1` - 结构化数据
  - `2` - 网站数据
  - `3` - [混合搜索](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#multi-data-stores)

### `GoogleCloudEnterpriseSearchRetriever` 的迁移指南

在之前的版本中，此检索器被称为 `GoogleCloudEnterpriseSearchRetriever`。

要更新到新检索器，请进行以下更改：

- 将导入从：`from langchain.retrievers import GoogleCloudEnterpriseSearchRetriever` -> `from langchain_google_community import VertexAISearchRetriever`。
- 将所有类引用从 `GoogleCloudEnterpriseSearchRetriever` -> `VertexAISearchRetriever`。

注意：在使用检索器时，如果你想从单个查询中获得自动化跟踪，你还可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## 实例化

### 为具有提取片段的**非结构化**数据配置和使用检索器

In [ ]:
from langchain_google_community import (
    VertexAIMultiTurnSearchRetriever,
    VertexAISearchRetriever,
)

PROJECT_ID = "<YOUR PROJECT ID>"  # Set to your Project ID
LOCATION_ID = "<YOUR LOCATION>"  # Set to your data store location
SEARCH_ENGINE_ID = "<YOUR SEARCH APP ID>"  # Set to your search app ID
DATA_STORE_ID = "<YOUR DATA STORE ID>"  # Set to your data store ID

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
)

In [ ]:
query = "What are Alphabet's Other Bets?"

result = retriever.invoke(query)
for doc in result:
    print(doc)

### 为**非结构化**数据配置并使用提取式问答的检索器

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    max_extractive_answer_count=3,
    get_extractive_answers=True,
)

result = retriever.invoke(query)
for doc in result:
    print(doc)

### 配置和使用检索器以获取**结构化**数据

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    engine_data_type=1,
)

result = retriever.invoke(query)
for doc in result:
    print(doc)

### 配置并使用检索器以**网站**数据与高级网站索引

### Configure and use the retriever for **website** data with Advanced Website Indexing

### 配置并使用检索器以**网站**数据与高级

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    max_extractive_answer_count=3,
    get_extractive_answers=True,
    engine_data_type=2,
)

result = retriever.invoke(query)
for doc in result:
    print(doc)

### 配置并使用检索器来处理**混合**数据

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    search_engine_id=SEARCH_ENGINE_ID,
    max_documents=3,
    engine_data_type=3,
)

result = retriever.invoke(query)
for doc in result:
    print(doc)

### 配置和使用检索器进行多轮搜索

[带后续问题的搜索](https://cloud.google.com/generative-ai-app-builder/docs/multi-turn-search) 基于生成式 AI 模型，与常规的非结构化数据搜索不同。

In [ ]:
retriever = VertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=LOCATION_ID, data_store_id=DATA_STORE_ID
)

result = retriever.invoke(query)
for doc in result:
    print(doc)

## 用法

遵循上面的示例，我们使用 `.invoke` 来发出单个查询。由于检索器是 Runnables，我们也可以使用 [Runnable 接口](/docs/concepts/runnables)中的任何方法，例如 `.batch`。

## 在链中使用

我们还可以将检索器整合到[链](/docs/how_to/sequence/)中，以构建更大的应用程序，例如简单的[RAG](/docs/tutorials/rag/)应用程序。为了演示目的，我们还实例化了一个 VertexAI 聊天模型。有关设置说明，请参阅相应的 Vertex[集成文档](/docs/integrations/chat/google_vertex_ai_palm/)。

In [ ]:
%pip install -qU langchain-google-vertexai

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_vertexai import ChatVertexAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

llm = ChatVertexAI(model_name="chat-bison", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(query)

## API 参考

如需了解 `VertexAISearchRetriever` 所有功能和配置的详细文档，请前往[API 参考](https://python.langchain.com/api_reference/google_community/vertex_ai_search/langchain_google_community.vertex_ai_search.VertexAISearchRetriever.html)。